<h3> <strong> 1. 실험 </strong> </h3> 

아래 명령을 이용해 학습을 시켜보자.

* python train.py --img 1024 --batch 8 --epochs 70 --data './data/labeling_dataset.yaml' --cfg './models/yolov5x.yaml' --weights './runs/train/YoloV5x_label_smoothing_Hyp_tuning/weights/best.pt' --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --label-smoothing 0.1 --name YoloV5x_labeling_pretrained --cache

이후 아래 명령을 이용해 test box에 대한 bbox를 생성해야 한다.

* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_labeling_pretrained/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_labeling_pretrained
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_labeling_pretrained/weights/best.pt' --augment --save-txt --save-conf --name YoloV5x_labeling_pretrained_IOU

In [4]:
import pandas as pd
import os

df = pd.DataFrame(index=[i for i in range(4871)], columns=['PredictionString', 'image_id'])
yolo_path = './../yolov5/runs/detect/YoloV5x_labeling_pretrained_IOU/labels'
for img_index in range(4871):
    image_id = str(img_index)
    while len(image_id)<4: image_id = '0'+image_id

    PredictionString = ''
    try: 
        txt_file = open(os.path.join(yolo_path, image_id+'.txt'), 'r')
        lines = txt_file.readlines()
    
        for line in lines:
            category, yolo_x, yolo_y, yolo_w, yolo_h, pred = (line.split())
            x_min = (float(yolo_x)-float(yolo_w)/2)*1024
            y_min = (float(yolo_y)-float(yolo_h)/2)*1024
            x_max = x_min+float(yolo_w)*1024
            y_max = y_min+float(yolo_h)*1024
            PredictionString += f'{category} {pred} {x_min} {y_min} {x_max} {y_max} '
        txt_file.close()
    except FileNotFoundError:
        pass
    
    df.iloc[img_index] = [PredictionString, 'test/'+image_id+'.jpg']

df.to_csv('./YoloV5x_labeling_pretrained_IOU.csv', index=False)

<h3> <strong> 2. 결과 </strong> </h3>

(1) Pseudo Labeling 적용: 0.5171 / 기존 0.5017 mAP에 비해 0.01 정도 향상됨. <br>
(2) IOU 감소시 성능이 향상될 때도, 떨어질 때도 있어서 다른 모델 제출을 위해 제출하지 않음.